# **Projek Akhir Praktikum Data Science**

> Analisis Sentimen Mengenai Vaksin COVID-19 Di Indonesia Menggunakan Metode Naive Bayes Classifier dan NLP Pada Sosial Media Twitter

**Oleh Kelompok 2 :**
1. Hazlan Muhammad Qodri (123190080) @hzlnqodrey
2. Elisia Dwi Rahayu (123190062) @elisiadwirahayu
3. Shania Septika Inayasari (123190055) @shaniainayasari

**Penjelasan Projek :**

Adapun pada penelitian menekankan kepada sentimen masyarakat terhadap mengenai vaksin COVID-19. Proses analisisnya akan dilakukan berdasarkan tweet yang menyertakan tagar vaksin dan pencarian di twitter dengan keyword vaksin covid 19.

## **1. Scraping Data from Twitter**

In [ ]:
%pip install snscrape

In [ ]:
import pandas as pd
import numpy as np
import snscrape.modules.twitter as sntwitter
import re


#### Query

In [ ]:
# Get All Covid Sentiment Data from January 1st, 2020 until November 1st, 2022
query = "covid since:2020-01-01 until:2022-11-01 lang:id"
limit = 2000 # limit 50k rows

In [ ]:
tweets = []

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets) == limit:
        break
    else:
        tweets.append([
            tweet.date,
            tweet.username,
            tweet.content
        ])

filename = 'tweets_covid_dataset_2k_raw_noindex.csv'
tweets_df = pd.DataFrame(tweets, columns=['Tanggal', 'Username', 'Text'])
tweets_df.to_csv(filename, index=False)
print('Scraping has completed!')

## **2. Wrangling Data** (Preprocessing)

In [ ]:
%pip install tweet-preprocessor
%pip install textblob
%pip install wordcloud
%pip install nltk

In [ ]:
import preprocessor as preproc
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import csv

In [101]:
# get data from dataset
data = pd.read_csv('https://raw.githubusercontent.com/hzlnqodrey/projek-akhir-prak-ds-sentimen-analisis-twitter-covid19-nlp-bayes/main/tweets_covid_dataset_2k_raw_noindex.csv')

In [102]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Tanggal   2000 non-null   object
 1   Username  2000 non-null   object
 2   Text      2000 non-null   object
dtypes: object(3)
memory usage: 47.0+ KB


In [103]:
data.sample(n=5)

,Tanggal,Username,Text
521,2022-10-31 14:05:38+00:00,_Kaumhalu,@dayatpiliang 2020 karena gabut covid+gak ada temen cerita
709,2022-10-31 12:30:37+00:00,aiyengood,I vote #straykids for Worldwide Fan's Choice Top 10 #MAMAVOTE\n\nbtw my dinner prepared by my mom. kena covid ni perut jd kecik ke cemana? nak perabih ni amek masa sejam. 🥲 https://t.co/ibMjxwphiI
396,2022-10-31 15:04:17+00:00,KIKOTAPASANDO1,RESUMEN COVID-19 PA SIMAN DI 31 DI OKTOBER 2022 https://t.co/7qUf5YKWqr
992,2022-10-31 10:33:33+00:00,tigisatnihs,"@zahwaisl Samaaaakkk.\nAku ngeh bgt tu pas jaman2 orang ga percaya covid dan saling fitnah nakes lah, pemerenta lah, elit global lah. Baru fitnah virus loh itu yg keliatan aja engga, belum dajjal yg bisa ngidupin orang mati."
912,2022-10-31 11:03:39+00:00,GreenandGold29,@MwendeCharles Time ya covid aliingia mitini


##### 1. Case Folding

In [104]:
data['Text'] = data['Text'].str.lower()

In [105]:
print('Case Folding Result : \n')
data.sample(n=5)

Case Folding Result : 



,Tanggal,Username,Text
1386,2022-10-31 07:53:08+00:00,yeeunohyeeun,"@foxgibbang benerrr jadi serba salah ㅠㅠ serem bgt deh ya. tahun2 sblmnya banyak yg meninggal krn covid. thn ini ya yg rusuh, tragedi tak terduga, trs musibah aduuh 😭😭😭"
1988,2022-10-31 04:17:12+00:00,mbahndi,@ardi_tama1 gila ya tuh kerumunan...ke mana satgas covid...
741,2022-10-31 12:16:46+00:00,Veteran_covid,"@yattie2022 @sean1724501685 @kompastv itu tokoh di channel youtube woko, calon mantune pak galino, koncone pak gendut, pacare tiwok."
1531,2022-10-31 07:07:07+00:00,harandist_,@mewmiawmew @kegblgnunfaedh saudaraku harusnya brgkt 2020 krn kena covid akhirnya kegeser sampai skrg blm berangkat\n\nsampai dia umroh awal tahun lalu\nentah kegeser berapa tahun
382,2022-10-31 15:12:05+00:00,HensemAdam1,@izzywafi ingatkan positive covid test result


In [106]:
data.isnull().sum()

Tanggal     0
Username    0
Text        0
dtype: int64

##### 2. Cleaning

In [107]:
def preprocessing_data(x):
    return preproc.clean(x)

data['Text'] = data['Text'].apply(preprocessing_data)


In [111]:
print('Cleaning Result : \n')
data.sample(n=5)

Cleaning Result : 



,Tanggal,Username,Text
444,2022-10-31 14:42:54+00:00,atiyahznddin,"terlalu banyak sijil online yang aku lupa link dia, anjay covid skang ni nak cari balik pun taktahu nak search apa"
318,2022-10-31 15:52:48+00:00,ainajaena,"nak tanya, rakyat malaysia tahukan yang sekarang da boleh bukak mask? sebab every where i go, mostly orang sekeliling i still pakai mask. bagus bagus, nampaknya rakyat kita masih aware dengan wabak covid-19. nah bunga untuk malaysians"
240,2022-10-31 16:38:13+00:00,shafkim95,kwn ajak keje part time kat satu mall ni mcm bagi free sample barang. today until khamis. tapi tiba2 tak jadi sbb ada staff kena covid. takde rezeki
1219,2022-10-31 09:06:36+00:00,DomiThaliaPutri,gileee replyan dokternya banyak banget konstipation belieber ckckckmulai dr yg ga percaya covid dan yg percaya bahwa vaksin itu isinya aneh2 bumi itu bentuknya apa ya?
74,2022-10-31 22:33:37+00:00,smartfm_medan,kementerian kesehatan (kemenkes) menargetkan implementasi platform saling pengakuan sertifikat kesehatan digital di negara-negara g20 terjadi pada kuartal i .kesepakatan ini memungkinkan bukti vaksinasi covid-19 digital mudah terverifikasi di negara-negara g20.semua


In [112]:
data['Text'].head(5)

0                                 dulu di barat upn ada warung kayuh bambai. bumbu habangnya sungguh banjar banar. semenjak tahun ke covid mereka tutup. padahal itu warung yang sangat mengobati kerinduan sama masakan banjar. nasi kuning dan soto banjarnya juara sekali
1                                                                                                                                                                                                                             segala sakit yg disalahin vaksin covid. lieur.
2    bukankah kamu dulu bagian rezim?, namun setelah terdepak dan sakit hati karena rezekinya hilang maka jadi sok kritis? dulu mengkritisi vaksin covid, namun saat divaksin pamer di medsos. ucapanmu tidak konsisten bro...semua karena sakit hati atas hilangnya rezeki.
3                                                                                                                                                                                   "aku kan gant

In [ ]:
# 

##### 3. Tokenizing

In [ ]:
def tokenize_data(x):
    return word_tokenize(x)

data['Text'] = data['Text'].apply(tokenize_data)

In [ ]:
print('Tokenizing Result : \n')
data.sample(n=5)

##### 4. Filtering

## **2.1 Stemming**

## **2.2 Menejermah Tweet Clean hasil Preproc ke Bahasa Inggris** (Translating)

## **3. Modelling Data** (Modeling and Training)

## **4. Klasifikasi Data dengan Naive Bayes** (Classification)